In [ ]:
// Import NuGet packages
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.*"
#r "nuget: Microsoft.Extensions.AI, 9.*-*"
#r "nuget: Microsoft.Extensions.DependencyInjection.Abstractions, 9.*"
#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"


In [ ]:
// Import namespaces
using System.ClientModel;
using System.IO;

using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.AI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using OpenAI;


In [ ]:
// Import OpenAI Configurations
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Development.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();

In [ ]:
// Create a Semantic Kernel instance with Azure OpenAI chat completions
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       endpoint: config["Azure:OpenAI:Endpoint"],
                       apiKey: config["Azure:OpenAI:Token"],
                       deploymentName: config["Azure:OpenAI:ModelName"])
                   .Build();

In [ ]:
// Import native plugin codebase
#!import Plugins/Appointment.cs
#!import Plugins/BookingsPlugin.cs


In [ ]:
// Import native plugin
kernel.Plugins.AddFromType<BookingsPlugin>();


In [ ]:
// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Start the conversation
string? input = null;


In [ ]:
// Enable function calling behaviour: Auto
var executionSettings = new OpenAIPromptExecutionSettings
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

In [ ]:
// Enable function calling behaviour: Required
var plugin = kernel.Plugins.SingleOrDefault(p => p.Name == "BookingsPlugin");
var function = plugin.SingleOrDefault(p => p.Name == "ListRestaurant");

var executionSettings = new OpenAIPromptExecutionSettings
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Required(functions: new[] { function })
};

In [ ]:
// Enable function calling behaviour: None
var executionSettings = new OpenAIPromptExecutionSettings
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.None()
};

In [ ]:
// Run assistant
ChatHistory chatHistory = [];

while (true)
{
    Console.Write("User > ");

    try
    {
        input = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("무엇을 도와드릴까요?");
    }
    catch (Exception ex)
    {
        Console.WriteLine("이용해 주셔서 감사합니다.");
        break;
    }

    chatHistory.AddUserMessage(input);

    // Get the result from the AI
    var result = await chatCompletionService.GetChatMessageContentAsync(chatHistory, executionSettings, kernel);

    // Print the result
    Console.WriteLine("Assistant > " + result);

    // Add the message from the agent to the chat history
    chatHistory.AddMessage(result.Role, result?.Content!);
}
